In [1]:
import os
path = "/Users/patricia/Documents/code/python-code/behavior-detection/src"
os.chdir(path)  # Muda o diretório para o nível anterior (a raiz do projeto)
print(os.getcwd())  # Verifique se agora está na raiz

/Users/patricia/Documents/code/python-code/behavior-detection/src


# Load data

In [2]:
from behavior.behavior_data_loader import BehaviorDataLoader

data_path = '../data/new_logs_labels.csv'

data = BehaviorDataLoader.load_data(data_path, delimiter=';')
data.head(5)

,id_log,aluno,grupo,num_dia,num_log,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,...,comportamento_off_task,comportamento_on_system,comportamento_indefinido,ultimo_comportamento,ultimo_comportamento_on_task,ultimo_comportamento_on_task_conversation,ultimo_comportamento_on_task_out,ultimo_comportamento_off_task,ultimo_comportamento_on_system,ultimo_comportamento_indefinido
0,8224,1,2,1,1,step_verification,0,0,1,0,...,0,0,0,?,0,0,0,0,0,1
1,527786,1,2,1,2,user_idle,1,0,0,2,...,0,0,0,ON TASK,1,0,0,0,0,0
2,527787,1,2,1,3,user_idle,0,0,0,4,...,0,0,0,ON TASK,1,0,0,0,0,0
3,527788,1,2,1,4,user_idle,0,0,0,6,...,0,0,0,ON TASK,1,0,0,0,0,0
4,527789,1,2,1,5,user_idle,0,0,0,8,...,0,0,0,ON TASK,1,0,0,0,0,0


In [3]:
from core.preprocessors.data_cleaner import DataCleaner

print("Valores da coluna 'comportamento' antes da remoção:", data['comportamento'].value_counts())

# Remove instances where 'comportamento' is '?'
data = DataCleaner.remove_instances_with_value(data, 'comportamento', '?')

print("\nValores da coluna 'comportamento' depois da remoção:", data['comportamento'].value_counts())

Valores da coluna 'comportamento' antes da remoção: comportamento
ON TASK                 3159
ON SYSTEM                907
OFF TASK                 629
ON TASK CONVERSATION     414
ON TASK OUT              380
?                         36
Name: count, dtype: int64

Valores da coluna 'comportamento' depois da remoção: comportamento
ON TASK                 3159
ON SYSTEM                907
OFF TASK                 629
ON TASK CONVERSATION     414
ON TASK OUT              380
Name: count, dtype: int64


In [4]:
data.head(5)

,id_log,aluno,grupo,num_dia,num_log,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,...,comportamento_off_task,comportamento_on_system,comportamento_indefinido,ultimo_comportamento,ultimo_comportamento_on_task,ultimo_comportamento_on_task_conversation,ultimo_comportamento_on_task_out,ultimo_comportamento_off_task,ultimo_comportamento_on_system,ultimo_comportamento_indefinido
0,8224,1,2,1,1,step_verification,0,0,1,0,...,0,0,0,?,0,0,0,0,0,1
1,527786,1,2,1,2,user_idle,1,0,0,2,...,0,0,0,ON TASK,1,0,0,0,0,0
2,527787,1,2,1,3,user_idle,0,0,0,4,...,0,0,0,ON TASK,1,0,0,0,0,0
3,527788,1,2,1,4,user_idle,0,0,0,6,...,0,0,0,ON TASK,1,0,0,0,0,0
4,527789,1,2,1,5,user_idle,0,0,0,8,...,0,0,0,ON TASK,1,0,0,0,0,0


In [5]:
## Select a subset of the data only for testing purposes

# Selecionar um subconjunto dos dados
# print("Tamanho do dataframe antes:", data.shape)
# data = data.sample(n=40, random_state=42)
# data.reset_index(drop=True, inplace=True)
# print("Tamanho do dataframe após:", data.shape)

# Pre-processing

## Remove unnecessary columns

In [6]:
# Removing columns related to IDs, emotions, personality and behaviors, because 
# we want to classify behaviors only by the students' interactions with the system
columns_to_remove_ids = ['id_log', 'grupo', 'num_dia', 'num_log']
columns_to_remove_emotions = [
    'estado_afetivo', 'estado_engajamento_concentrado', 
    'estado_confusao', 'estado_frustracao', 'estado_tedio', 'estado_indefinido', 
    'ultimo_estado_afetivo', 'ultimo_engajamento_concentrado', 'ultimo_confusao', 
    'ultimo_frustracao', 'ultimo_tedio', 'ultimo_estado_indefinido'
]
columns_to_remove_personality = [
    'traco_amabilidade_fator', 'traco_extrovercao_fator', 'traco_conscienciosidade_fator', 
    'traco_abertura_fator', 'traco_neuroticismo_fator', 'traco_amabilidade_cat', 
    'traco_extrovercao_cat', 'traco_conscienciosidade_cat', 'traco_abertura_cat', 
    'traco_neuroticismo_cat']

columns_to_remove_behaviors = [
    'comportamento_on_task', 'comportamento_on_task_conversation', 'comportamento_on_task_out',
    'comportamento_off_task', 'comportamento_on_system', 'comportamento_indefinido',
    'ultimo_comportamento', 'ultimo_comportamento_on_task', 'ultimo_comportamento_on_task_conversation',
    'ultimo_comportamento_on_task_out', 'ultimo_comportamento_off_task', 'ultimo_comportamento_on_system',
    'ultimo_comportamento_indefinido'
]

columns_to_remove = columns_to_remove_ids + \
        columns_to_remove_emotions + \
        columns_to_remove_personality + \
        columns_to_remove_behaviors

cleaned_data = DataCleaner.remove_columns(data, columns_to_remove)


In [7]:
cleaned_data.head(5)

,aluno,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,num_click_acumulado,num_click_passo,num_click_eq,type_step_verification,...,misc_OI_Dv_Plus_Sb_total,misc_EqSec_Distrib_MtTerm_total,misc_OI_Mt_Minus_Mt_Plus_total,misc_OI_Mt_Minus_Mt_Minus_total,misc_OI_Dv_Plus_Ad_total,misc_EqPrim_Mt_Inc_total,misc_EqPrim_Dv_Inc_total,misc_OI_Dv_Minus_Dv_Minus_total,misc_EqSec_OpFrac_MMC_MtNumerador_total,comportamento
0,1,step_verification,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,ON TASK
1,1,user_idle,1,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ON TASK
2,1,user_idle,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ON TASK
3,1,user_idle,0,0,0,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ON TASK
4,1,user_idle,0,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ON TASK


In [8]:
# Preenche valores ausentes no DataFrame X com a string 'missing'.

cleaned_data = cleaned_data.fillna('missing')

## Split data by student level into training and test datasets

In [9]:
from core.preprocessors.data_splitter import DataSplitter

train_data, test_data = DataSplitter.split_by_student_level(cleaned_data, test_size=0.2, column_name='aluno')

In [10]:
# removing the 'aluno' column from the data after splitting into train and test sets

# Remover 'aluno' do conjunto de treinamento
cleaned_data = DataCleaner.remove_columns(train_data, ['aluno'])

# Remover 'aluno' do conjunto de teste
cleaned_data = DataCleaner.remove_columns(test_data, ['aluno'])

## Split data into Features (X) and Target (y)

In [11]:
from core.preprocessors.data_splitter import DataSplitter

# Conjunto de treinamento
X_train, y_train = DataSplitter.split_into_x_y(train_data, 'comportamento')

# Conjunto de teste
X_test, y_test = DataSplitter.split_into_x_y(test_data, 'comportamento')

In [12]:
print("Primeiras 5 instâncias de y_train:")
print(y_train[:5])

print("\nPrimeiras 5 instâncias de y_test:")
print(y_test[:5])

Primeiras 5 instâncias de y_train:
0    ON TASK
1    ON TASK
2    ON TASK
3    ON TASK
4    ON TASK
Name: comportamento, dtype: object

Primeiras 5 instâncias de y_test:
1450    ON TASK
1451    ON TASK
1452    ON TASK
1453    ON TASK
1454    ON TASK
Name: comportamento, dtype: object


## Encoding variables

### Encoding true labels (y)

In [13]:
import importlib
from core.preprocessors import column_selector, data_encoder
from behavior import behavior_data_encoder

# Recarregar o módulo para garantir que as alterações sejam aplicadas
importlib.reload(column_selector)
importlib.reload(data_encoder)
importlib.reload(behavior_data_encoder)

<module 'behavior.behavior_data_encoder' from '/Users/patricia/Documents/code/python-code/behavior-detection/src/behavior/behavior_data_encoder.py'>

In [14]:
# Encoding y_train and y_test
from behavior.behavior_data_encoder import BehaviorDataEncoder

# Codificar y_train
y_train = BehaviorDataEncoder.encode_y(y_train)

# Codificar y_test
y_test = BehaviorDataEncoder.encode_y(y_test)

### Encoding features (X)

In [15]:
# Pré-processar X_train
X_encoder = BehaviorDataEncoder(num_classes=5)
X_encoder.fit(X_train)

X_train = X_encoder.transform(X_train)

# Pré-processar X_test usando o mesmo preprocessor
X_test = X_encoder.transform(X_test)

# Balanceamento dos dados

In [16]:
from core.preprocessors.data_balancer import DataBalancer

data_balancer = DataBalancer()
X_train, y_train = data_balancer.apply_smote(X_train, y_train)

In [18]:
from collections import Counter

print(f"Resampled dataset shape: {Counter(y_train)}")

Resampled dataset shape: Counter({2: 2519, 1: 2519, 4: 2519, 0: 2519, 3: 2519})


# Treinamento dos Modelos usando Otimização Bayesiana (BayesSearchCV)

In [ ]:
from core.training.bayesian_optimization_training import BayesianOptimizationTraining
cv = 3
n_iter = 100
n_jobs = 4  # Number of processors to be used in the execution: -1 to use all processors

# Choose a scoring metric
scoring_metric = 'roc_auc_ovr'  # Possible values: 'f1_macro', 'balanced_accuracy', 'roc_auc_ovr', etc.

training = BayesianOptimizationTraining()

trained_models = training.train_model(
    X_train, y_train, n_iter=n_iter, cv=cv, scoring=scoring_metric, n_jobs=n_jobs
)

Training and evaluating Decision Tree with BayesianOptimization and pca
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 2/3] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=19, feature_selection__n_components=10;, score=0.792 total time=   0.3s
[CV 1/3] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=19, feature_selection__n_components=10;, score=0.755 total time=   0.4s
[CV 3/3] END classifier__max_depth=5, classifier__min_samples_leaf=8, classifier__min_samples_split=19, feature_selection__n_components=10;, score=0.721 total time=   0.4s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 2/3] END classifier__max_depth=30, classifier__min_samples_leaf=9, classifier__min_samples_split=7, feature_selection__n_components=50;, score=0.888 total time=   1.2s
[CV 3/3] END classifier__max_depth=30, classifier__min_samples_leaf=9, classifier__min_samples_split=7, feature_selection_

/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [None, 10, 2, 40] before, using random point [5, 10, 19, 30]
  warnings.warn(


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END classifier__max_depth=5, classifier__min_samples_leaf=10, classifier__min_samples_split=19, feature_selection__n_components=30;, score=0.789 total time=   0.5s
[CV 2/3] END classifier__max_depth=5, classifier__min_samples_leaf=10, classifier__min_samples_split=19, feature_selection__n_components=30;, score=0.786 total time=   0.5s
[CV 3/3] END classifier__max_depth=5, classifier__min_samples_leaf=10, classifier__min_samples_split=19, feature_selection__n_components=30;, score=0.791 total time=   0.5s


/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [None, 10, 2, 40] before, using random point [30, 10, 9, 50]
  warnings.warn(


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END classifier__max_depth=30, classifier__min_samples_leaf=10, classifier__min_samples_split=9, feature_selection__n_components=50;, score=0.870 total time=   1.0s
[CV 2/3] END classifier__max_depth=30, classifier__min_samples_leaf=10, classifier__min_samples_split=9, feature_selection__n_components=50;, score=0.889 total time=   0.9s
[CV 3/3] END classifier__max_depth=30, classifier__min_samples_leaf=10, classifier__min_samples_split=9, feature_selection__n_components=50;, score=0.868 total time=   0.9s


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END classifier__max_depth=10, classifier__min_samples_leaf=6, classifier__min_samples_split=14, feature_selection__n_components=30;, score=0.840 total time=   0.6s
[CV 2/3] END classifier__max_depth=10, classifier__min_samples_leaf=6, classifier__min_samples_split=14, feature_selection__n_components=30;, score=0.864 total time=   0.5s
[CV 3/3] END classifier__max_depth=10, classifier__min_samples_leaf=6, classifier__min_samples_split=14, feature_selection__n_components=30;, score=0.844 total time=   0.6s


/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [None, 10, 2, 40] before, using random point [30, 3, 12, 30]
  warnings.warn(


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END classifier__max_depth=30, classifier__min_samples_leaf=3, classifier__min_samples_split=12, feature_selection__n_components=30;, score=0.846 total time=   0.6s
[CV 2/3] END classifier__max_depth=30, classifier__min_samples_leaf=3, classifier__min_samples_split=12, feature_selection__n_components=30;, score=0.875 total time=   0.8s
[CV 3/3] END classifier__max_depth=30, classifier__min_samples_leaf=3, classifier__min_samples_split=12, feature_selection__n_components=30;, score=0.866 total time=   0.7s


/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [None, 10, 2, 30] before, using random point [5, 2, 19, 40]
  warnings.warn(


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END classifier__max_depth=5, classifier__min_samples_leaf=2, classifier__min_samples_split=19, feature_selection__n_components=40;, score=0.788 total time=   0.7s
[CV 2/3] END classifier__max_depth=5, classifier__min_samples_leaf=2, classifier__min_samples_split=19, feature_selection__n_components=40;, score=0.795 total time=   0.6s
[CV 3/3] END classifier__max_depth=5, classifier__min_samples_leaf=2, classifier__min_samples_split=19, feature_selection__n_components=40;, score=0.787 total time=   0.5s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END classifier__max_depth=None, classifier__min_samples_leaf=10, classifier__min_samples_split=2, feature_selection__n_components=20;, score=0.844 total time=   0.5s
[CV 2/3] END classifier__max_depth=None, classifier__min_samples_leaf=10, classifier__min_samples_split=2, feature_selection__n_components=20;, score=0.876 total time=   0.5s
[CV 3/3] END cla

# Avaliação dos Modelos

In [ ]:
from core.evaluation.evaluation import Evaluation  

feature_names = X_train.columns  # Assumindo que os nomes das características são as colunas
class_metrics_results, avg_metrics_results = Evaluation.evaluate_all_models(trained_models, X_train, y_train, X_test, y_test, feature_names)

# Geração dos Relatórios

In [ ]:
from core.logging.report_formatter import ReportFormatter
from core.logging.file_manager import FileManager

directory = "../output/"

# Gerar relatório textual a partir dos resultados de avaliação
text_report = ReportFormatter.generate_text_report(class_metrics_results, avg_metrics_results)

# Imprimir ou salvar o relatório
FileManager.save_file_with_timestamp(text_report, "bayesian_optimization_report.txt", directory)

# Gerar DataFrame detalhado dos relatórios por classe
class_report_df = ReportFormatter.generate_class_report_dataframe(class_metrics_results)

# Gerar DataFrame resumido dos relatórios de métricas médias
avg_metrics_report_df = ReportFormatter.generate_avg_metrics_report_dataframe(avg_metrics_results)

# Salvar os DataFrames como arquivos CSV, se necessário
FileManager.save_csv_file_with_timestamp(class_report_df, "class_report.csv", directory)
FileManager.save_csv_file_with_timestamp(avg_metrics_report_df, "avg_metrics_report.csv", directory)


# Salvando os modelos em arquivos para recuperação

In [ ]:
from core.logging.model_manager import ModelManager

# Caminhos
model_dir = "../models/"

# Salvar todos os modelos
saved_models = ModelManager.save_all_models(trained_models, model_dir)
print("Modelos salvos:", saved_models)